# **Named entity recognition (NER):**
# - Find the entities (such as persons, locations, or organizations) in a sentence.
# - This can be formulated as attributing a label to each token by having one class per entity and one class for “no entity.”
# - It falls under token_classification and this project done by using Pytorch framework

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Installing packages
!pip install transformers datasets tokenizers seqeval -q

In [3]:
# !pip install tokenizers
# !pip install accelerate -U

In [1]:
import datasets
import numpy as np
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification

In [2]:
data = datasets.load_dataset("conll2003")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [3]:
# Partitions of data
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [4]:
# Train data
data['train']

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 14041
})

In [5]:
# Train data Feature
data['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [6]:
# Display the tags available in data
data["train"].features["ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [7]:
# data description
data['train'].description

'The shared task of CoNLL-2003 concerns language-independent named entity recognition. We will concentrate on\nfour types of named entities: persons, locations, organizations and names of miscellaneous entities that do\nnot belong to the previous three groups.\n\nThe CoNLL-2003 shared task data files contain four columns separated by a single space. Each word has been put on\na separate line and there is an empty line after each sentence. The first item on each line is a word, the second\na part-of-speech (POS) tag, the third a syntactic chunk tag and the fourth the named entity tag. The chunk tags\nand the named entity tags have the format I-TYPE which means that the word is inside a phrase of type TYPE. Only\nif two phrases of the same type immediately follow each other, the first word of the second phrase will have tag\nB-TYPE to show that it starts a new phrase. A word with tag O is not part of a phrase. Note the dataset uses IOB2\ntagging scheme, whereas the original dataset uses 

## Retreiving BERT tokenizer from hugging face

In [8]:
# Initialize the tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [9]:
# Testing how tokenizer is working
ex_text = data['train'][0]
tokenized_input = tokenizer(ex_text['tokens'], is_split_into_words=True)
# calling tokenized_input
tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])

word_ids = tokenized_input.word_ids()
word_ids

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]

**NOTE :** None is like start of sentence and end of sentence




In [10]:
# word embedding using pretrained BERT model
tokenized_input

{'input_ids': [101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

**Note :**
- The above format is important as it acts as input to the model       
- [cls-101, sep-102]

In [11]:
# Actual Tokens
tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])
print('Tokens :',tokens)

word_ids = tokenized_input.word_ids() # integer encoding
print("unique_num_word : ", word_ids)    # word embedding (BERT)

Tokens : ['[CLS]', 'eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.', '[SEP]']
unique_num_word :  [None, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]


In [12]:
data['train'][0]['ner_tags']

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [13]:
tokens

['[CLS]',
 'eu',
 'rejects',
 'german',
 'call',
 'to',
 'boycott',
 'british',
 'lamb',
 '.',
 '[SEP]']

In [14]:
len(tokens)

11

**NOTE :**

Problem of Sub-Token - The input ids returned by the tokenizer are longer than the lists of labels our dataset contain.


In [15]:

print('Len of tokens :',len(data['train'][0]['ner_tags']))
print('Len of Tokenizers :',len(tokens))

Len of tokens : 9
Len of Tokenizers : 11


### The below function tokenize_and_align_labels does 2 jobs:
- Set –100 as the label for these special tokens and the subwords we wish to mask during training

- Mask the subword representations after the first subword

Then we align the labels with the token ids using the strategy we picked:

In [16]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token.
        previous_word_idx = None
        label_ids = []
        # Special tokens like `` and `<\s>` are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token
                label_ids.append(label[word_idx])
            else:
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # mask the subword representations after the first subword

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

### Testing on One Example

In [17]:
data['train'][0:1]

{'id': ['0'],
 'tokens': [['EU',
   'rejects',
   'German',
   'call',
   'to',
   'boycott',
   'British',
   'lamb',
   '.']],
 'pos_tags': [[22, 42, 16, 21, 35, 37, 16, 21, 7]],
 'chunk_tags': [[11, 21, 11, 12, 21, 22, 11, 12, 0]],
 'ner_tags': [[3, 0, 7, 0, 0, 0, 7, 0, 0]]}

In [18]:
a = tokenize_and_align_labels(data['train'][0:1])
a

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]]}

### So before applying the tokenize_and_align_labels() the tokenized_input has 3 keys:
    input_ids
    token_type_ids
    attention_mask

### But after applying tokenize_and_align_labels() we have an extra key - 'labels'


In [19]:
# Calling the labels
for token, label in zip(tokenizer.convert_ids_to_tokens(a['input_ids'][0]),a['labels'][0]):
  print(f"{token:-<40}{label}")

[CLS]------------------------------------100
eu--------------------------------------3
rejects---------------------------------0
german----------------------------------7
call------------------------------------0
to--------------------------------------0
boycott---------------------------------0
british---------------------------------7
lamb------------------------------------0
.---------------------------------------0
[SEP]------------------------------------100


## Now Applying on Entire Data

In [20]:
tokenized_datasets = data.map(tokenize_and_align_labels, batched = True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [21]:
tokenized_datasets['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'input_ids': [101,
  7327,
  19164,
  2446,
  2655,
  2000,
  17757,
  2329,
  12559,
  1012,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]}

**NOTE :** Now the data is preprocessed.

In [22]:
# Defining the model
# num_labels = 9 --> conll['train'].features["ner_tags"] output is --> ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels = 9)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**NOTE :** Some arguments must set to the BERT model as we are fine tuning BERT model.

In [23]:
# !pip install accelerate -U

In [36]:
#Define training args
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
                        "test-ner",
                        evaluation_strategy = "epoch",
                        learning_rate=2e-5,
                        per_device_train_batch_size=16,
                        per_device_eval_batch_size=16,
                        num_train_epochs=1,
                        weight_decay=0.01,
                        )

In [37]:
data_collator = DataCollatorForTokenClassification(tokenizer)

## Evaluation metric For BERT: "seqeval"

In [40]:
metric = datasets.load_metric("seqeval")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [41]:
example = data['train'][0]

In [42]:
label_list = data['train'].features['ner_tags'].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [43]:
for i in example['ner_tags']:
  print(i)

3
0
7
0
0
0
7
0
0


In [44]:
labels = [label_list[i] for i in example['ner_tags']]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [45]:
metric.compute(predictions = [labels], references = [labels])

{'MISC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

## Compute metrics :
This compute_metrics() function first takes the argmax of the logits to convert them to predictions (as usual, the logits and the probabilities are in the same order, so we don’t need to apply the softmax). Then we have to convert both labels and predictions from integers to strings. We remove all the values where the label is -100, then pass the results to the metric.compute() method:

In [46]:
def compute_metrics(eval_preds):
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)

    return {
          "precision": results["overall_precision"],
          "recall": results["overall_recall"],
          "f1": results["overall_f1"],
          "accuracy": results["overall_accuracy"],
  }

## Training

In [47]:
trainer = Trainer(
   model,
   args,
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets["validation"],
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics
)

In [48]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.086300,0.062031,0.918282,0.932767,0.925468,0.982906


TrainOutput(global_step=878, training_loss=0.07939217465342041, metrics={'train_runtime': 159.318, 'train_samples_per_second': 88.132, 'train_steps_per_second': 5.511, 'total_flos': 342221911376202.0, 'train_loss': 0.07939217465342041, 'epoch': 1.0})

## Save the model

In [49]:
# Save model

model.save_pretrained("NER_BERT_model")

In [50]:
# Save tokenizer

tokenizer.save_pretrained("NER_BERT_tokenizer")

('NER_BERT_tokenizer/tokenizer_config.json',
 'NER_BERT_tokenizer/special_tokens_map.json',
 'NER_BERT_tokenizer/vocab.txt',
 'NER_BERT_tokenizer/added_tokens.json',
 'NER_BERT_tokenizer/tokenizer.json')

## Now should define 2 things (i) id2label (ii) label2id to pass to configuration

In [51]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [52]:
id2label

{'0': 'O',
 '1': 'B-PER',
 '2': 'I-PER',
 '3': 'B-ORG',
 '4': 'I-ORG',
 '5': 'B-LOC',
 '6': 'I-LOC',
 '7': 'B-MISC',
 '8': 'I-MISC'}

In [53]:
label2id

{'O': '0',
 'B-PER': '1',
 'I-PER': '2',
 'B-ORG': '3',
 'I-ORG': '4',
 'B-LOC': '5',
 'I-LOC': '6',
 'B-MISC': '7',
 'I-MISC': '8'}

## Loading the model & prediction


In [54]:
import json

In [56]:
config = json.load(open("/content/NER_BERT_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("/content/NER_BERT_model/config.json","w"))

In [58]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("/content/NER_BERT_model")

In [59]:
from transformers import pipeline

In [60]:
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)


example = "Bill Gates is the Founder of Microsoft"

ner_results = nlp(example)

print(ner_results)

[{'entity': 'B-PER', 'score': 0.9907238, 'index': 1, 'word': 'bill', 'start': 0, 'end': 4}, {'entity': 'I-PER', 'score': 0.9924077, 'index': 2, 'word': 'gates', 'start': 5, 'end': 10}, {'entity': 'B-ORG', 'score': 0.9037774, 'index': 7, 'word': 'microsoft', 'start': 29, 'end': 38}]


# Dataset link : https://huggingface.co/datasets/conll2003
# Model Link : https://huggingface.co/bert-base-uncased
# Reference link : https://huggingface.co/course/chapter7/2